In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(42)

In [3]:
df = pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [4]:
df.shape

(6000, 785)

In [5]:
## Data Loading and batching layer
## Declaring Architecture: 784 -> 128(relu) -> 64(relu) -> 10(sigmoid)
## Training the data 
## Evaluating the data

In [11]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:].values, df.iloc[:, 0].values, test_size=0.2, random_state=42)

# Normalizing the data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [12]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long) # why long? 
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    

batch_size = 32
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
## Defining the neural network architecture

class SimpleNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            # Pytorch has by-default softmax in the last layer
            # nn.Softmax(dim=1) is not needed
            # nn.Softmax(dim=1)
        )
    def forward(self, x):
        # Forward pass through the network
        return self.model(x)

In [16]:
# Declaring the training process parameters
num_epochs = 100
learning_rate = 0.1

# Instantiate the model
model = SimpleNN(train_dataset.data.shape[1])
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# Training the model

for epoch in range(num_epochs):
    epoch_loss = 0
    for batch_features, batch_labels in train_loader:
        # Forward pass
        outputs = model(batch_features)
        loss = loss_function(outputs, batch_labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    average_loss = epoch_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}')


Epoch [1/100], Loss: 1.3161
Epoch [2/100], Loss: 0.7693
Epoch [3/100], Loss: 0.6599
Epoch [4/100], Loss: 0.5805
Epoch [5/100], Loss: 0.5305
Epoch [6/100], Loss: 0.4906
Epoch [7/100], Loss: 0.4745
Epoch [8/100], Loss: 0.4311
Epoch [9/100], Loss: 0.4174
Epoch [10/100], Loss: 0.4045
Epoch [11/100], Loss: 0.3877
Epoch [12/100], Loss: 0.3550
Epoch [13/100], Loss: 0.3582
Epoch [14/100], Loss: 0.3432
Epoch [15/100], Loss: 0.3280
Epoch [16/100], Loss: 0.3154
Epoch [17/100], Loss: 0.3080
Epoch [18/100], Loss: 0.2805
Epoch [19/100], Loss: 0.2822
Epoch [20/100], Loss: 0.2650
Epoch [21/100], Loss: 0.2571
Epoch [22/100], Loss: 0.2707
Epoch [23/100], Loss: 0.2551
Epoch [24/100], Loss: 0.2319
Epoch [25/100], Loss: 0.2194
Epoch [26/100], Loss: 0.2256
Epoch [27/100], Loss: 0.2067
Epoch [28/100], Loss: 0.2051
Epoch [29/100], Loss: 0.2039
Epoch [30/100], Loss: 0.1869
Epoch [31/100], Loss: 0.2096
Epoch [32/100], Loss: 0.1753
Epoch [33/100], Loss: 0.1773
Epoch [34/100], Loss: 0.1700
Epoch [35/100], Loss: 0

In [18]:
model.eval()  # Set the model to evaluation mode

correct = 0
total = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        outputs = model(batch_features)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

In [19]:
total_accuracy = 100 * correct / total
print(f'Accuracy of the model on the test set: {total_accuracy:.2f}%')

Accuracy of the model on the test set: 83.67%
